In [1]:
import os
import json
from typing import Dict, Any, Optional, Tuple
import requests

In [2]:
ENCODED_KEY  = os.environ.get("ODCLOUD_ENCODED_KEY",  "%2FP5AhAmvjuBQBRALCoeppdhwG2TdfyZiiQNcc%2BZkkbdec3MKMsgRs3GZUa%2BPuCCtdEsTd7DtbbgmLjCTitOhSA%3D%3D")
DECODED_KEY  = os.environ.get("ODCLOUD_DECODED_KEY",  "/P5AhAmvjuBQBRALCoeppdhwG2TdfyZiiQNcc+Zkkbdec3MKMsgRs3GZUa+PuCCtdEsTd7DtbbgmLjCTitOhSA==")

BASE = "https://api.odcloud.kr/api"

In [3]:
def _request(path: str, params: Dict[str, Any], method: str = "GET") -> Tuple[Optional[dict], requests.Response]:
    """
    ODcloud는 보통 querystring에 serviceKey를 전달합니다.
    인코딩 키 우선 → 실패 시 디코딩 키로 재시도합니다.
    """
    url = f"{BASE}{path}"
    # 1) 인코딩 키
    p1 = dict(params)
    p1["serviceKey"] = ENCODED_KEY
    r = requests.get(url, params=p1, timeout=30) if method == "GET" else requests.post(url, params=p1, timeout=30)
    if r.ok:
        try:
            return r.json(), r
        except Exception:
            pass

    # 2) 디코딩 키
    p2 = dict(params)
    p2["serviceKey"] = DECODED_KEY
    r2 = requests.get(url, params=p2, timeout=30) if method == "GET" else requests.post(url, params=p2, timeout=30)
    if r2.ok:
        try:
            return r2.json(), r2
        except Exception:
            pass

    # 3) 실패 시 원본 응답 반환
    return None, r2

In [4]:
def pretty_keys(d: Any, max_items: int = 10) -> None:
    """상위 키/샘플 몇 개만 빠르게 확인"""
    if isinstance(d, dict):
        print(f"[dict] keys({len(d.keys())}):", list(d.keys())[:20])
    elif isinstance(d, list):
        print(f"[list] len={len(d)}")
        for i, item in enumerate(d[:max_items]):
            print(f"  - [{i}] type={type(item).__name__} keys={list(item.keys())[:20] if isinstance(item, dict) else 'N/A'}")
    else:
        print(type(d).__name__, d)

def pick_service_id(item: dict) -> Optional[str]:
    """목록 아이템에서 svcId(또는 유사 키)를 추정"""
    for k in ("svcId", "serviceId", "SERV_ID", "svcID", "svc_id"):
        if k in item and isinstance(item[k], str):
            return item[k]
    # id 비슷한 키 휴리스틱
    for k, v in item.items():
        if isinstance(v, str) and "id" in k.lower():
            return v
    return None

def save_json(name: str, obj: Any) -> None:
    with open(name, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)
    print(f"📁 saved -> {name}")

def fetch_service_list(page: int = 1, per_page: int = 10, filters: Optional[Dict[str, Any]] = None) -> dict:
    """
    목록 API: /gov24/v3/serviceList
    - 기본 파라미터: page, perPage
    - 선택: lifeArray, trgterIndvdlArray, intrsThemaArray, age, ctpvNm, sggNm, srchKeyCode, searchWrd, arrgOrd 등(사용 시 filters에 넣기)
    """
    params = {"page": page, "perPage": per_page, "returnType": "JSON"}
    if filters:
        params.update(filters)
    data, resp = _request("/gov24/v3/serviceList", params)
    if data is None:
        raise RuntimeError(f"serviceList 호출 실패: HTTP {resp.status_code} - {resp.text[:300]}")
    return data

def fetch_service_detail(svc_id: str) -> dict:
    params = {"svcId": svc_id, "returnType": "JSON"}
    data, resp = _request("/gov24/v3/serviceDetail", params)
    if data is None:
        raise RuntimeError(f"serviceDetail 호출 실패(svcId={svc_id}): HTTP {resp.status_code} - {resp.text[:300]}")
    return data

def fetch_support_conditions(svc_id: str) -> dict:
    params = {"svcId": svc_id, "returnType": "JSON"}
    data, resp = _request("/gov24/v3/supportConditions", params)
    if data is None:
        raise RuntimeError(f"supportConditions 호출 실패(svcId={svc_id}): HTTP {resp.status_code} - {resp.text[:300]}")
    return data

In [5]:
if __name__ == "__main__":
    # (선택) 목록 필터 예시 — 필요 없으면 지우세요.
    # filters = {
    #     "lifeArray": "004",            # 생애주기(예: 청년)
    #     "trgterIndvdlArray": "050",    # 대상(예: 저소득)
    #     "intrsThemaArray": "030",      # 관심테마(예: 생활지원)
    #     "ctpvNm": "서울특별시",
    #     "sggNm": "중구",
    #     "srchKeyCode": "003",          # 검색키(예: 서비스명)
    #     "searchWrd": "",               # 검색어
    #     "arrgOrd": "001",              # 정렬
    # }
    filters = None

    print("\n=== 1) 서비스 목록 호출 ===============================")
    lst = fetch_service_list(page=1, per_page=10, filters=filters)
    pretty_keys(lst)
    # ODcloud 표준 응답에 보통 data 배열이 들어있습니다.
    data_list = lst.get("data", lst)  # 혹시 최상위가 바로 리스트로 오는 경우 대비
    pretty_keys(data_list)

    if not isinstance(data_list, list) or len(data_list) == 0:
        raise SystemExit("목록이 비어 있습니다. 필터를 조정해 다시 시도하세요.")

    # 첫 항목 훑어보기
    first_item = data_list[0]
    print("\n[목록 첫 항목 일부 내용]")
    for k in list(first_item.keys())[:30]:
        print(f"  {k}: {first_item[k]}")

    svc_id = pick_service_id(first_item)
    if not svc_id:
        raise SystemExit("svcId(서비스 식별자)를 찾지 못했습니다. 목록 항목의 키를 확인해 주세요.")
    print(f"\n→ 선택된 svcId: {svc_id}")

    save_json("service_list_sample.json", lst)

    print("\n=== 2) 서비스 상세 호출 ===============================")
    detail = fetch_service_detail(svc_id)
    pretty_keys(detail)
    # 상세 응답 안의 주요 키 훑기
    if isinstance(detail, dict):
        print("\n[상세 상위 키/값(일부)]")
        for k in list(detail.keys())[:30]:
            v = detail[k]
            summary = f"{type(v).__name__}({len(v)} items)" if isinstance(v, (list, dict)) else str(v)[:120]
            print(f"  {k}: {summary}")
    save_json(f"service_detail_{svc_id}.json", detail)

    print("\n=== 3) 지원조건 호출 =================================")
    cond = fetch_support_conditions(svc_id)
    pretty_keys(cond)
    # 지원조건 내부 구조 미리보기
    if isinstance(cond, dict):
        print("\n[지원조건 상위 키/값(일부)]")
        for k in list(cond.keys())[:30]:
            v = cond[k]
            summary = f"{type(v).__name__}({len(v)} items)" if isinstance(v, (list, dict)) else str(v)[:120]
            print(f"  {k}: {summary}")
    save_json(f"support_conditions_{svc_id}.json", cond)

    print("\n✅ 완료: JSON 파일 3개가 현재 폴더에 저장되었습니다.")


=== 1) 서비스 목록 호출 ===============================
[dict] keys(6): ['currentCount', 'data', 'matchCount', 'page', 'perPage', 'totalCount']
[list] len=10
  - [0] type=dict keys=['등록일시', '부서명', '사용자구분', '상세조회URL', '서비스ID', '서비스명', '서비스목적요약', '서비스분야', '선정기준', '소관기관명', '소관기관유형', '소관기관코드', '수정일시', '신청기한', '신청방법', '전화문의', '접수기관', '조회수', '지원내용', '지원대상']
  - [1] type=dict keys=['등록일시', '부서명', '사용자구분', '상세조회URL', '서비스ID', '서비스명', '서비스목적요약', '서비스분야', '선정기준', '소관기관명', '소관기관유형', '소관기관코드', '수정일시', '신청기한', '신청방법', '전화문의', '접수기관', '조회수', '지원내용', '지원대상']
  - [2] type=dict keys=['등록일시', '부서명', '사용자구분', '상세조회URL', '서비스ID', '서비스명', '서비스목적요약', '서비스분야', '선정기준', '소관기관명', '소관기관유형', '소관기관코드', '수정일시', '신청기한', '신청방법', '전화문의', '접수기관', '조회수', '지원내용', '지원대상']
  - [3] type=dict keys=['등록일시', '부서명', '사용자구분', '상세조회URL', '서비스ID', '서비스명', '서비스목적요약', '서비스분야', '선정기준', '소관기관명', '소관기관유형', '소관기관코드', '수정일시', '신청기한', '신청방법', '전화문의', '접수기관', '조회수', '지원내용', '지원대상']
  - [4] type=dict keys=['등록일시', '부서명', '사용자구분', '상세조회URL', '서비스I